# Preparación de entorno

In [1]:

from qiskit.utils import algorithm_globals

algorithm_globals.random_seed = 123456

In [2]:
import torch

_ = torch.manual_seed(123456)  # suppress output

In [3]:
import numpy as np

num_dim = 1
num_discrete_values = 8
num_qubits = num_dim * int(np.log2(num_discrete_values))

In [4]:
import torch
from qiskit.utils import algorithm_globals
import numpy as np
from scipy.stats import norm

algorithm_globals.random_seed = 123456
_ = torch.manual_seed(123456)  # suppress output

# Fijar la cantidad de dimensiones, la cantidad de valores discretos y calcular la cantidad de qubits requeridos
num_dim = 1  # Cambiamos a 1 dimensión
num_discrete_values = 8  # Esto puede ajustarse según tus necesidades
num_qubits = num_dim * int(np.log2(num_discrete_values))

# Crear una distribución normal 1D
mean = 0.0  # Media
std_dev = 1.0  # Desviación estándar
coords = np.linspace(mean - 2 * std_dev, mean + 2 * std_dev, num_discrete_values)
prob_data = norm.pdf(coords, loc=mean, scale=std_dev)
prob_data = prob_data / np.sum(prob_data)

# Visualizar la distribución normal 1D
import matplotlib.pyplot as plt

plt.plot(coords, prob_data, marker='o')
plt.xlabel('Valor')
plt.ylabel('Probabilidad')
plt.title('Distribución Normal 1D')
plt.grid(True)
plt.show()


In [5]:
from qiskit import QuantumCircuit
from qiskit.circuit.library import EfficientSU2

qc = QuantumCircuit(num_qubits)
qc.h(qc.qubits)

ansatz = EfficientSU2(num_qubits, reps=1)#probar de modificar, para comparar mas resultados
qc.compose(ansatz, inplace=True)

In [6]:
qc.decompose().draw("mpl")

In [7]:
from qiskit.primitives import Sampler

shots = 10000 #
sampler = Sampler(options={"shots": shots, "seed": algorithm_globals.random_seed})

In [8]:
from qiskit_machine_learning.connectors import TorchConnector
from qiskit_machine_learning.neural_networks import SamplerQNN


def create_generator() -> TorchConnector:
    qnn = SamplerQNN(
        circuit=qc,
        sampler=sampler,
        input_params=[],
        weight_params=qc.parameters,
        sparse=False,
    )

    initial_weights = algorithm_globals.random.random(qc.num_parameters)
    return TorchConnector(qnn, initial_weights)

In [9]:
from torch import nn


class Discriminator(nn.Module):
    def __init__(self, input_size):
        super(Discriminator, self).__init__()

        self.linear_input = nn.Linear(input_size, 20)
        self.leaky_relu = nn.LeakyReLU(0.2)
        self.linear20 = nn.Linear(20, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, input: torch.Tensor) -> torch.Tensor:
        x = self.linear_input(input)
        x = self.leaky_relu(x)
        x = self.linear20(x)
        x = self.sigmoid(x)
        return x


In [10]:
generator = create_generator()
discriminator = Discriminator(num_dim)

In [11]:
def adversarial_loss(input, target, w):
    bce_loss = target * torch.log(input) + (1 - target) * torch.log(1 - input)
    weighted_loss = w * bce_loss
    total_loss = -torch.sum(weighted_loss)
    return total_loss


In [12]:
from torch.optim import Adam

lr = 0.01  # learning rate
b1 = 0.7  # first momentum parameter
b2 = 0.999  # second momentum parameter

generator_optimizer = Adam(generator.parameters(), lr=lr, betas=(b1, b2), weight_decay=0.005)
discriminator_optimizer = Adam(
    discriminator.parameters(), lr=lr, betas=(b1, b2), weight_decay=0.005
)

In [13]:
from IPython.display import clear_output


def plot_training_progress():
    # we don't plot if we don't have enough data
    if len(generator_loss_values) < 2:
        return

    clear_output(wait=True)
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(18, 9))

    # Generator Loss
    ax1.set_title("Loss")
    ax1.plot(generator_loss_values, label="generator loss", color="royalblue")
    ax1.plot(discriminator_loss_values, label="discriminator loss", color="magenta")
    ax1.legend(loc="best")
    ax1.set_xlabel("Iteration")
    ax1.set_ylabel("Loss")
    ax1.grid()

    # Relative Entropy
    ax2.set_title("Relative entropy")
    ax2.plot(entropy_values)
    ax2.set_xlabel("Iteration")
    ax2.set_ylabel("Relative entropy")
    ax2.grid()

    with torch.no_grad():
        generated_probabilities = generator(torch.tensor([])).numpy()  # Generar muestras utilizando el generador

    # Generar CDF de las probabilidades generadas
    gen_prob_cdf = np.cumsum(generated_probabilities)   
    ax3.plot(coords, gen_prob_cdf, label='Generated CDF', color='royalblue')
    ax3.plot(coords, norm.cdf(coords, loc=mean, scale=std_dev), label='True PDF', color='magenta')
    ax3.set_title('Comparación de CDF Generada y Verdadera')
    ax3.set_xlabel('Valor')
    ax3.set_ylabel('Probabilidad Acumulada')
    ax3.legend()
    ax3.grid()
 

    # Dibujar la PDF generada en un gráfico 2D
    ax4.plot(coords, generated_probabilities, label='Generated PDF', color='royalblue')
    ax4.plot(coords, norm.pdf(coords, loc=mean, scale=std_dev), label='True PDF', color='magenta')
    ax4.set_title('Comparación de PDF Generada y Verdadera')
    ax4.set_xlabel('Valor')
    ax4.set_ylabel('Densidad de Probabilidad')
    ax4.legend()
    ax4.grid(True)


    plt.show()

Fit in 439.31 sec


In [14]:
import time
from scipy.stats import multivariate_normal, entropy

n_epochs = 200

num_qnn_outputs = num_discrete_values**num_dim

generator_loss_values = []
discriminator_loss_values = []
entropy_values = []

start = time.time()
for epoch in range(n_epochs):

    valid = torch.ones(num_qnn_outputs, 1, dtype=torch.float)
    fake = torch.zeros(num_qnn_outputs, 1, dtype=torch.float)

    # Configure input
    real_dist = torch.tensor(prob_data, dtype=torch.float).reshape(-1, 1)

    disc_value = discriminator(real_dist)  # Cambiado a 'real_dist'

    # Generate data
    gen_dist = generator(torch.tensor([])).reshape(-1, 1)

    # Train generator
    generator_optimizer.zero_grad()
    generator_loss = adversarial_loss(disc_value, valid, gen_dist)

    # store for plotting
    generator_loss_values.append(generator_loss.detach().item())

    generator_loss.backward(retain_graph=True)
    generator_optimizer.step()

    # Train Discriminator
    discriminator_optimizer.zero_grad()

    real_loss = adversarial_loss(disc_value, valid, real_dist)
    fake_loss = adversarial_loss(disc_value, fake, gen_dist.detach())
    discriminator_loss = (real_loss + fake_loss) / 2

    # Store for plotting
    discriminator_loss_values.append(discriminator_loss.detach().item())

    discriminator_loss.backward()
    discriminator_optimizer.step()

    entropy_value = entropy(gen_dist.detach().squeeze().numpy(), prob_data)
    entropy_values.append(entropy_value)

    plot_training_progress()

elapsed = time.time() - start
print(f"Fit in {elapsed:0.2f} sec")
